In [1]:
# -*- coding: utf8 -*-

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
conf = SparkConf().setAppName("hospitalDisease")
sc = SparkContext(conf=conf)


In [3]:
sqlContext = SQLContext(sc)

### load入院诊断数据 

In [15]:
import pandas as pd
#829177
diaPath = "hospitalData/admissionDiagnosis.csv"
pd_dia = pd.read_csv(diaPath,  encoding='utf8').drop_duplicates()
pd_dia.count()
pd_dia.head(2)

/opt/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id,ar_id,disease_code,disease,district_code,node_code,diagnosis_date,diagnosis_type_code,diagnosis_type_name,diagnosis_basis
0,0,37170049542543-X2016000151,37170049542543-X2016000151,NaN,肺癌,371724.0,371700,20161223,3.0,入院初步诊断,咳嗽，痰中带血16月余。左肺呼吸音低，右肺呼吸音粗，未闻及啰音。
1,1,240894_R04.200,240894,R04.201,咯血,3713.0,371300,20170331,3.0,NaN,1.患者孔庆忠，男，76岁。既往高血压病史。现因“反复咳嗽、间断咯血2年，加重2天”入院。 ...


### load入院记录数据

In [17]:
import pandas as pd
#pd_poc = pd.read_csv("datasets/hospitaldisease-201305after.csv",  encoding='utf8').drop_duplicates()
pd_poc = pd.read_csv("hospitalData/hospitaldisease-201305after_match20171129.csv",  encoding='utf8').drop_duplicates()
#print(pd_poc.count)

/opt/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
pd_poc.head(2)

,NAME,ID_NUM,SEX_CODE,SEX_NAME,DISEASE_CODE,DISEASE,ADMISSION_DATE,Code,DiseaseICD10,SecondCode,SecondType,FirstCode,FirstType
0,徐志刚,222403198304026016,1,男,T93.501,下肢肌肉损伤后遗症,20150402 13:50:02,T93.501,下肢肌肉损伤后遗症,T90-T98,损伤、中毒和外因的其他后果的后遗症,S00-T98,损伤、中毒和外因的某些其它后果
1,张俊哲,370123199706200034,1,男,T93.501,下肢肌肉损伤后遗症,20150701 15:08:57,T93.501,下肢肌肉损伤后遗症,T90-T98,损伤、中毒和外因的其他后果的后遗症,S00-T98,损伤、中毒和外因的某些其它后果


### load ICD-10编码标准

In [10]:
import pandas as pd
pd_icd10 = pd.read_excel("./datasets/ICD-10.xlsx", sheet_name="icdsecondNew")
pd_icd10_new = pd_icd10[["Code","Disease","SecondCode","SecondType","FirstCode","FirstType"]]
pd_icd10_new = pd_icd10_new.rename(columns={'Disease': 'DiseaseICD10'})

In [19]:
firstCodeSet = set(pd_icd10_new["FirstCode"].tolist())
print(firstCodeSet)

{'Q00-Q99', 'KOO-K93', 'G00-G99', 'H00-H59', 'U00-U99', 'V01-Y98', 'F01-F99', 'L00-L99', 'N00-N99', 'A00-B99', 'E00-E90', 'I00-I99', 'Z00-Z99', 'D50-D89', 'R00-R99', 'J00-J99', 'M00-M99', 'H60-H95', 'P00-P96', 'C00-D48', 'O00-O99', 'S00-T98'}


In [11]:
def doListFormat(diseaseList):
    for i in range(len(diseaseList)):
        if pd.isnull(diseaseList[i]):
            diseaseList[i] = ""
        diseaseList[i] = diseaseList[i].strip()#.encode("utf8")
    return diseaseList

In [13]:
def list2Save(disList, f):
    disList = doListFormat(disList)
    disLen = len(disList)
    disNum = int(disLen/100)
    #print("%d %d" %(disLen, disNum))
    i=0
    for i in range(disNum):
        tmpStr = ""
        #print(disList[100*i:100*(i+1)])
        tmpStr = " ".join(disList[100*i:100*(i+1)])
        f.write(tmpStr+"\n")
    tmpStr = ""
    tmpStr = " ".join(disList[100*i:disLen])
    f.write(tmpStr+"\n")
    #f.flush()
    #f.close()

In [24]:
from random import shuffle
def word2vecTrainMatch2txt(f,pd_poc):

    for item in firstCodeSet:
        pd_tmp = pd_poc.where(pd_poc.FirstCode==item).dropna()
        tmpList = pd_tmp["DiseaseICD10"].tolist()
        #print(tmpList[0:100])
        shuffle(tmpList)
        #print(len(tmpList))
        #print(tmpList[0:100])
        list2Save(tmpList,f)

### 构造word2vec训练集合，先将同code的一级分类下的数据随机打散组合，然后随机组合成100个疾病

In [25]:
f1 = open("word2vecDatasets/word2vecTrain1.txt","w")
word2vecTrainMatch2txt(f1,pd_poc)
f1.close()
f2 = open("word2vecDatasets/word2vecTrain2.txt","w")
word2vecTrainMatch2txt(f2,pd_icd10_new)
f2.close()

In [26]:
import re
def doWordFormat(word):
    #word=u"128我(我是我）"
    word = word.strip()
    word = word.replace("（","(")
    word = word.replace("）",")")
    
    fil0 = re.compile('\d+.' )
    word = fil0.sub('', word)
    
    fil1 = re.compile('\((.*?)\)' )
    wordFilter = fil1.sub(' ',word)#.decode("utf8")
    #filtrate = re.compile('[.。、，,：:?？+\(【】\)!！]')#146051
    #filtrate = re.compile('[a-z.。、，,：:?？+\(【】\)!！]') #146055
    #filtrate = re.compile('[A-Z.。、，,：:?？+\(【】\)!！]')#150598
    #filtrate = re.compile('[a-zA-Z.。、，,：:?？+\(【】\)!！]')#150834
    #filtrate = re.compile('[0-9.。、，,：:?？+\(【】\)!！]')#146056
    filtrate = re.compile('[a-zA-Z0-9.。、，,：:?？+\(【】\)!！]')#150892
    #filtrate = re.compile('[a-zA-Z0-9.。、，,：:?？+\(【】\)!！]')#非中文
    filtered_str = filtrate.sub(' ', wordFilter)#.decode("utf8")#replace

    return filtered_str.strip()

In [29]:
pd_pocNo = pd.read_csv("hospitalData/hospitaldisease-201305after_nomatch20171129.csv",  encoding='utf8').drop_duplicates()
pd_pocNo["DISEASENew"] = pd_pocNo["DISEASE"].apply(doWordFormat)

pd_pocNoJoin = pd.merge(pd_pocNo,pd_icd10_new,left_on='DISEASE_CODE',right_on='Code')
pd_pocNoJoin.head(2)

,NAME,ID_NUM,SEX_CODE,SEX_NAME,DISEASE_CODE,DISEASE,ADMISSION_DATE,DISEASENew,Code,DiseaseICD10,SecondCode,SecondType,FirstCode,FirstType
0,丁万安,370102193012312913,1,男,L02.402,下肢皮肤脓肿、疖和痈,20160116 15:15:33,下肢皮肤脓肿 疖和痈,L02.402,臂脓肿,L00-L08,皮肤和皮下组织的感染,L00-L99,皮肤和皮下组织病
1,孟繁熹,370102194507070324,2,女,L02.402,下肢皮肤脓肿、疖和痈,20151126 09:58:49,下肢皮肤脓肿 疖和痈,L02.402,臂脓肿,L00-L08,皮肤和皮下组织的感染,L00-L99,皮肤和皮下组织病


### 不匹配的数据，先按照code一致的组合，然后将所有的随机组合

In [31]:

def word2vecTrainNoMatchtxt(f,pd_poc):
    
    for item in firstCodeSet:
        #print(item)
        pd_tmp = pd_poc.where(pd_poc.FirstCode==item).dropna()
        tmpList = pd_tmp["DISEASENew"].tolist()
        shuffle(tmpList)
        #print(len(tmpList))
        #print(tmpList[0:100])
        list2Save(tmpList,f)
f1 = open("word2vecDatasets/word2vecTrain3.txt","w")
word2vecTrainNoMatchtxt(f1,pd_pocNoJoin)
f1.close()
def word2vecTrainNoMatch2txt(f,pd_poc):
    tmpList = pd_poc["DISEASENew"].tolist()
    #print(tmpList[0:100])
    shuffle(tmpList)
    print(len(tmpList))
    list2Save(tmpList,f)
f2 = open("word2vecDatasets/word2vecTrain4.txt","w")
word2vecTrainNoMatch2txt(f2,pd_pocNo)
f2.close()

1535126
